In [1]:
from google.colab import drive
drive.mount('/content/drive/',force_remount=True)

Mounted at /content/drive/


In [2]:
%cd "/content/drive/My Drive/KCAMF/Attacks"

/content/drive/My Drive/KCAMF/Attacks


# **TA Pre**

In [3]:
import pandas as pd
from sklearn.utils import shuffle

ta = pd.read_csv("/content/drive/My Drive/KCAMF/Attacks/dataset/TripAdvisor/Data_TripAdvisor_v21.csv")
ta = shuffle(ta)
ta.head(10)

,UserID,ItemID,Rating,TripType
3567,95AEE0D9E39636C5CFDF572B2EE547FD,84131,5,COUPLES
2444,198C193CFF7CDE1B61DE684FE9C0C7A3,85283,5,SOLO
5503,25D11772090601050CBC58AFFFAFE339,208755,5,FAMILY
1247,10BE6590174588A1D9B12261E99112AD,1144098,5,FRIENDS
1498,00D673CA0747712BD29890CB31E3C58D,1456416,5,FAMILY
10043,3D0F83D200FB537E2353EB8FDA070FBE,294839,4,COUPLES
11972,84C799DE5BCD26CF7E5C32DF3BD32599,89377,3,FAMILY
12094,A76A1570C4CBABBDDFB4F3F4078692CE,2280220,3,BUSINESS
5179,2CEFB076C1F40AB702CADD7FC6E37862,189078,5,COUPLES
10769,02EE1392650D0D3CF79A9CB2C0A8F221,91674,4,COUPLES


In [4]:
mapping = {item:i for i, item in enumerate(ta["UserID"].unique())}
ta["UserID"] = ta["UserID"].apply(lambda x: mapping[x])

mapping = {item:i for i, item in enumerate(ta["ItemID"].unique())}
ta["ItemID"] = ta["ItemID"].apply(lambda x: mapping[x])
ta.head()

,UserID,ItemID,Rating,TripType
3567,0,0,5,COUPLES
2444,1,1,5,SOLO
5503,2,2,5,FAMILY
1247,3,3,5,FRIENDS
1498,4,4,5,FAMILY


In [5]:
len(ta.UserID.unique()), len(ta.ItemID.unique())

(2371, 2269)

In [6]:
ta[['Rating']] = ta[['Rating']].astype({'Rating': float})
ta.dtypes

UserID        int64
ItemID        int64
Rating      float64
TripType     object
dtype: object

In [7]:
import numpy as np

msk = np.random.rand(len(ta)) < 0.8

ta_train = ta[msk]
ta_test = ta[~msk]

In [8]:
ta_train.head()

,UserID,ItemID,Rating,TripType
2444,1,1,5.0,SOLO
5503,2,2,5.0,FAMILY
1247,3,3,5.0,FRIENDS
1498,4,4,5.0,FAMILY
10043,5,5,4.0,COUPLES


In [9]:
len(ta_train.UserID.unique()), len(ta_train.ItemID.unique())

(2369, 2130)

In [10]:
len(ta_test.UserID.unique()), len(ta_test.ItemID.unique())

(1746, 1286)

In [11]:
len(ta_train), len(ta_test)

(11276, 2899)

In [12]:
ta_test.head()

,UserID,ItemID,Rating,TripType
3567,0,0,5.0,COUPLES
11972,6,6,3.0,FAMILY
5179,8,8,5.0,COUPLES
769,10,10,5.0,SOLO
8737,13,13,4.0,BUSINESS


In [13]:
fname = 'dataset/TripAdvisor/TA_train.csv'
ta_train.to_csv(fname, index=False)

fname = 'dataset/TripAdvisor/TA_test.csv'
ta_test.to_csv(fname, index=False)

# **Warming up**

In [14]:
!ls

attackDataset	    dataset		 MyTAAttacks.ipynb
attack.py	    GANattack.py	 randomAttack.py
averageAttack.py    getTargetsItem.py	 shillingfile.py
bandwagonAttack.py  LDos_LH.csv		 unorganizedMaliciousAttacks.py
config		    main.py
config.py	    MyLDosAttacks.ipynb


In [15]:
import os
path = '/content/drive/My Drive/KCAMF/Attacks/config/config.conf'
config = {}
print(path)
if not os.path.exists(path):
  print('config file is not found!')
  raise IOError
with open(path) as f:
  for ind,line in enumerate(f):
      if line.strip()!='':
          try:
              key,value=line.strip().split('=')
              config[key]=value
          except ValueError:
              print('config file is not in the correct format! Error Line:%d'%(ind))


/content/drive/My Drive/KCAMF/Attacks/config/config.conf


In [16]:
import os.path
class Config(object):
    def __init__(self,fileName):
        self.config = {}
        self.readConfiguration(fileName)

    def __getitem__(self, item):
        if not self.contains(item):
            print('parameter '+item+' is invalid!')
            exit(-1)
        return self.config[item]

    def getOptions(self,item):
        if not self.contains(item):
            print('parameter '+item+' is invalid!')
            exit(-1)
        return self.config[item]

    def contains(self,key):
        return self.config.__contains__(key)

    def readConfiguration(self,fileName):
        path = fileName
        print(path)
        if not os.path.exists(path):
            print('config file is not found!')
            raise IOError
        with open(path) as f:
            for ind,line in enumerate(f):
                if line.strip()!='':
                    try:
                        key,value=line.strip().split('=')
                        self.config[key]=value
                    except ValueError:
                        print('config file is not in the correct format! Error Line:%d'%(ind))

class LineConfig(object):
    def __init__(self,content):
        self.line = content.strip().split(' ')
        self.options = {}
        self.mainOption = False
        if self.line[0] == 'on':
            self.mainOption = True
        elif self.line[0] == 'off':
            self.mainOption = False
        for i,item in enumerate(self.line):
            if (item.startswith('-') or item.startswith('--')) and  not item[1:].isdigit():
                ind = i+1
                for j,sub in enumerate(self.line[ind:]):
                    if (sub.startswith('-') or sub.startswith('--')) and  not sub[1:].isdigit():
                        ind = j
                        break
                    if j == len(self.line[ind:])-1:
                        ind=j+1
                        break
                try:
                    self.options[item] = ' '.join(self.line[i+1:i+1+ind])
                except IndexError:
                    self.options[item] = 1


    def __getitem__(self, item):
        if not self.contains(item):
            print('parameter '+item+' is invalid!')
            exit(-1)
        return self.options[item]

    def getOption(self,key):
        if not self.contains(key):
            print('parameter '+key+' is invalid!')
            exit(-1)
        return self.options[key]

    def isMainOn(self):
        return self.mainOption

    def contains(self,key):
        return self.options.__contains__(key)




In [17]:
import os.path
from os import makedirs,remove
from re import compile,findall,split
#from config import LineConfig
from collections import defaultdict
class FileIO(object):
    def __init__(self):
        pass

    def writeFile(dir,file,content,op = 'w'):
        if not os.path.exists(dir):
            os.makedirs(dir)
        if type(content)=='str':
            with open(dir + file, op) as f:
                f.write(content)
        else:
            with open(dir+file,op) as f:
                f.writelines(content)

    def deleteFile(filePath):
        if os.path.exists(filePath):
            remove(filePath)

    def loadDataSet(conf, file, bTest=False):
        trainingData = defaultdict(dict)
        testData = defaultdict(dict)
        ratingConfig = LineConfig(conf['ratings.setup'])
        if not bTest:
            print('loading training data...')
        else:
            print('loading test data...')
        with open(file) as f:
            ratings = f.readlines()
        # ignore the headline
        if ratingConfig.contains('-header'):
            ratings = ratings[1:]
        # order of the columns
        order = ratingConfig['-columns'].strip().split()

        for lineNo, line in enumerate(ratings):
            items = split(' |,|\t', line.strip())
            if not bTest and len(order) < 3:
                print('The rating file is not in a correct format. Error: Line num %d' % lineNo)
                exit(-1)    
            try:
                userId = items[int(order[0])]
                itemId = items[int(order[1])]
                if bTest and len(order)<3:
                    rating = 1 #default value
                else:
                    rating  = items[int(order[2])]

            except ValueError:
                print('Error! Have you added the option -header to the rating.setup?')
                exit(-1)
            if not bTest:
                trainingData[userId][itemId]=float(rating)
            else:
                testData[userId][itemId] = float(rating)
        if not bTest:
            return trainingData
        else:
            return testData

    def loadRelationship(conf, filePath):
        socialConfig = LineConfig(conf['social.setup'])
        relation = []
        print('loading social data...')
        with open(filePath) as f:
            relations = f.readlines()
            # ignore the headline
        if socialConfig.contains('-header'):
            relations = relations[1:]
        # order of the columns
        order = socialConfig['-columns'].strip().split()
        if len(order) <= 2:
            print('The social file is not in a correct format.')
        for lineNo, line in enumerate(relations):
            items = split(' |,|\t', line.strip())
            if len(order) < 2:
                print('The social file is not in a correct format. Error: Line num %d' % lineNo)
                exit(-1)
            userId1 = items[int(order[0])]
            userId2 = items[int(order[1])]
            if len(order) < 3:
                weight = 1
            else:
                weight = float(items[int(order[2])])
            relation.append([userId1, userId2, weight])
        return relation


    def loadLabels(filePath):
        labels = {}
        with open(filePath) as f:
            for line in f:
                items = split(' |,|\t', line.strip())
                labels[items[0]] = items[1]
        return labels

In [18]:
import os.path
from os import makedirs,remove
from re import compile,findall,split
from collections import defaultdict
class FileIO(object):
    def __init__(self):
        pass

    def writeFile(dir,file,content,op = 'w'):
        if not os.path.exists(dir):
            os.makedirs(dir)
        if type(content)=='str':
            with open(dir + file, op) as f:
                f.write(content)
        else:
            with open(dir+file,op) as f:
                f.writelines(content)

    def deleteFile(filePath):
        if os.path.exists(filePath):
            remove(filePath)

    def loadDataSet(conf, file, bTest=False):
        trainingData = defaultdict(dict)
        testData = defaultdict(dict)
        ratingConfig = LineConfig(conf['ratings.setup'])
        if not bTest:
            print('loading training data...')
        else:
            print('loading test data...')
        with open(file) as f:
            ratings = f.readlines()
        # ignore the headline
        if ratingConfig.contains('-header'):
            ratings = ratings[1:]
        # order of the columns
        order = ratingConfig['-columns'].strip().split()

        for lineNo, line in enumerate(ratings):
            items = split(' |,|\t', line.strip())
            if not bTest and len(order) < 3:
                print('The rating file is not in a correct format. Error: Line num %d' % lineNo)
                exit(-1)    
            try:
                userId = items[int(order[0])]
                itemId = items[int(order[1])]
                if bTest and len(order)<3:
                    rating = 1 #default value
                else:
                    rating  = items[int(order[2])]

            except ValueError:
                print('Error! Have you added the option -header to the rating.setup?')
                exit(-1)
            if not bTest:
                trainingData[userId][itemId]=float(rating)
            else:
                testData[userId][itemId] = float(rating)
        if not bTest:
            return trainingData
        else:
            return testData

    def loadRelationship(conf, filePath):
        socialConfig = LineConfig(conf['social.setup'])
        relation = []
        print('loading social data...')
        with open(filePath) as f:
            relations = f.readlines()
            # ignore the headline
        if socialConfig.contains('-header'):
            relations = relations[1:]
        # order of the columns
        order = socialConfig['-columns'].strip().split()
        if len(order) <= 2:
            print('The social file is not in a correct format.')
        for lineNo, line in enumerate(relations):
            items = split(' |,|\t', line.strip())
            if len(order) < 2:
                print('The social file is not in a correct format. Error: Line num %d' % lineNo)
                exit(-1)
            userId1 = items[int(order[0])]
            userId2 = items[int(order[1])]
            if len(order) < 3:
                weight = 1
            else:
                weight = float(items[int(order[2])])
            relation.append([userId1, userId2, weight])
        return relation

    def loadLabels(filePath):
        labels = {}
        with open(filePath) as f:
            for line in f:
                items = split(' |,|\t', line.strip())
                labels[items[0]] = items[1]
        return labels

# **Attack Class**

In [19]:
from collections import defaultdict
import numpy as np
import random
import os
from os.path import abspath

class Attack(object):
    def __init__(self,conf):
        self.config = Config(conf)
        self.userProfile = FileIO.loadDataSet(self.config,self.config['ratings'])
        self.itemProfile = defaultdict(dict)
        self.attackSize = float(self.config['attackSize'])
        self.fillerSize = float(self.config['fillerSize'])
        self.selectedSize = float(self.config['selectedSize'])
        self.targetCount = int(self.config['targetCount'])
        self.targetScore = float(self.config['targetScore'])
        self.threshold = float(self.config['threshold'])
        self.minCount = int(self.config['minCount'])
        self.maxCount = int(self.config['maxCount'])
        self.minScore = float(self.config['minScore'])
        self.maxScore = float(self.config['maxScore'])
        self.outputDir = self.config['outputDir']
        if not os.path.exists(self.outputDir):
            os.makedirs(self.outputDir)
        for user in self.userProfile:
            for item in self.userProfile[user]:
                self.itemProfile[item][user] = self.userProfile[user][item]
        self.spamProfile = defaultdict(dict)
        self.spamItem = defaultdict(list) #items rated by spammers
        self.targetItems = []
        self.itemAverage = {}
        self.getAverageRating()
        self.selectTarget()
        self.startUserID = 0

    # def selectTarget(count = 20):
    #     pass
    # def reload(self):
    #     self.userProfile, self.itemProfile = loadRatings(self.config['ratings'])
    #     self.spamProfile = defaultdict(dict)
    #     self.spamItem = defaultdict(list)

    def getAverageRating(self):
        for itemID in self.itemProfile:
            li = self.itemProfile[itemID].values()
            self.itemAverage[itemID] = float(sum(li)) / len(li)


    def selectTarget(self,):
        print('Selecting target items...')
        #print('-')*80
        print('Target item       Average rating of the item')
        itemList = list(self.itemProfile.keys())
        sorted(itemList)
        targets = []
        '''
        with open('targets.txt') as f:
            content = f.readlines()
            for item in content:
                item = item.strip('\n')
                targets.append(item)
        #targets = ['3863','1388','1920','2084','204','2088','8861','99112','291','33836','1241','3705','5106','7367','46530','2402','102','2088','2709','1848']

        #targets = []
        #haha = []
        '''
        haha = []
        while len(self.targetItems) < self.targetCount:
            target = np.random.randint(len(itemList)) #generate a target order at random

            if len(self.itemProfile[str(itemList[target])]) < self.maxCount and len(self.itemProfile[str(itemList[target])]) > self.minCount \
                    and str(itemList[target]) not in self.targetItems \
                    and self.itemAverage[str(itemList[target])] <= self.threshold:
                self.targetItems.append(str(itemList[target]))
                haha.append(itemList[target])
                targets.append(itemList[target] + '\n')
                print(str(itemList[target]),'                  ',self.itemAverage[str(itemList[target])])
        
        for item in targets:
            self.targetItems.append(item)

        print(self.targetItems)
        
        
    def getFillerItems(self):
        mu = int(self.fillerSize*len(self.itemProfile))
        sigma = int(0.1*mu)
        markedItemsCount = abs(int(round(random.gauss(mu, sigma))))
        markedItems = np.random.randint(len(self.itemProfile), size=markedItemsCount)
        return markedItems.tolist()

    def insertSpam(self,startID=0):
        pass

    def loadTarget(self,filename):
        with open(filename) as f:
            for line in f:
                self.targetItems.append(line.strip())

    def generateLabels(self,filename):
        labels = []
        path = self.outputDir + filename
        with open(path,'w') as f:
            for user in self.spamProfile:
                labels.append(user+' 1\n')
            for user in self.userProfile:
                labels.append(user+' 0\n')
            f.writelines(labels)
        print('User profiles have been output to '+abspath(self.config['outputDir'])+'.')

    def generateProfiles(self,filename):
        ratings = []

        path = self.outputDir+filename
        with open(path, 'w') as f:
            for user in self.userProfile:
                for item in self.userProfile[user]:
                    ratings.append(user+' '+item+' '+str(self.userProfile[user][item])+'\n')

            for user in self.spamProfile:
                for item in self.spamProfile[user]:
                    ratings.append(user + ' ' + item + ' ' + str(self.spamProfile[user][item])+'\n')
            f.writelines(ratings)
        print('User labels have been output to '+abspath(self.config['outputDir'])+'.')
    
    def generateSpamProfiles(self,filename):
        #num_injected = int(len(self.userProfile)*self.attackSize * self.targetCount * len(self.getFillerItems()))
        num_injected = 20000
        #print(num_injected)
        ratings = np.zeros((num_injected, 3))
        #print(len(ratings))
        i = 0
        path = self.outputDir+filename
        for user in self.spamProfile:
          for item in self.spamProfile[user]:
            ratings[i][0] = int(user)
            ratings[i][1] = int(item)
            ratings[i][2] = self.spamProfile[user][item]
            #print(i)
            i = i + 1
            #ratings.append(user + "," + item + "," + str(self.spamProfile[user][item]))
        
        #print(ratings)
        ratings_df = pd.DataFrame(ratings)
        ratings_df.to_csv(path, header=False, index=False)

        '''
        with open(path, 'w') as f:
            for user in self.spamProfile:
                for item in self.spamProfile[user]:
                    ratings.append(user + "," + item + "," + 
                                   str(self.spamProfile[user][item]))
                    ratings.append("\n")
            f.writelines(ratings)
        '''
        print('Spam profiles have been output to '+abspath(self.config['outputDir'])+'.')



# **Random Attack**

In [20]:
import random
import numpy as np

class RandomAttack(Attack):
    def __init__(self,conf):
        super(RandomAttack, self).__init__(conf)


    def insertSpam(self,startID=0):
        print('Modeling random attack...')
        itemList = list(self.itemProfile.keys())
        
        if startID == 0:
            self.startUserID = len(self.userProfile)
        else:
            self.startUserID = startID

        for i in range(int(len(self.userProfile)*self.attackSize)):
            #fill 装填项目
            fillerItems = self.getFillerItems()
            #print(fillerItems)
            for item in fillerItems:
                self.spamProfile[str(self.startUserID)][str(itemList[item])] = random.randint(self.minScore,self.maxScore)

            #print(self.spamProfile)
            #target 目标项目
            #print(self.targetCount)
            for j in range(self.targetCount):
                #print(len(self.targetItems))
                target = np.random.randint(len(self.targetItems))
                self.spamProfile[str(self.startUserID)][self.targetItems[target]] = self.targetScore
                self.spamItem[str(self.startUserID)].append(self.targetItems[target])
            self.startUserID += 1
        
        print(self.spamProfile)


In [21]:
attack = RandomAttack('/content/drive/My Drive/KCAMF/Attacks/config/config.conf')
attack.insertSpam()
attack.generateProfiles('Rprofiles.csv')

/content/drive/My Drive/KCAMF/Attacks/config/config.conf
loading training data...
Selecting target items...
Target item       Average rating of the item
1466                    3.375
647                    3.3333333333333335
276                    3.4285714285714284
716                    3.5
675                    3.4285714285714284
666                    3.5
1284                    3.1666666666666665
1507                    3.3333333333333335
305                    3.142857142857143
86                    3.2
['1466', '647', '276', '716', '675', '666', '1284', '1507', '305', '86', '1466\n', '647\n', '276\n', '716\n', '675\n', '666\n', '1284\n', '1507\n', '305\n', '86\n']
Modeling random attack...
defaultdict(<class 'dict'>, {'2369': {'843': 3, '2077': 2, '601': 1, '1320': 4, '68': 2, '398': 2, '1467': 4, '566': 3, '757': 1, '1668': 5, '711': 5, '2158': 2, '1649': 2, '1112': 1, '1332': 1, '1371': 3, '1187': 5, '1521': 1, '1971': 4, '1210': 3, '963': 1, '976': 3, '666': 5.0, '647\n': 5.

# **Average Attack**

In [22]:
import random
import numpy as np

class AverageAttack(Attack):
    def __init__(self,conf):
        super(AverageAttack, self).__init__(conf)

    def insertSpam(self,startID=0):
        print('Modeling average attack...')
        itemList = list(self.itemProfile.keys())
        if startID == 0:
            self.startUserID = len(self.userProfile) + 1
        else:
            self.startUserID = startID

        for i in range(int(len(self.userProfile)*self.attackSize)):
            #fill 
            fillerItems = self.getFillerItems()
            for item in fillerItems:
                self.spamProfile[str(self.startUserID)][str(itemList[item])] = round(self.itemAverage[str(itemList[item])], 2)
            
            #target 
            for j in range(self.targetCount):
                target = np.random.randint(len(self.targetItems))
                self.spamProfile[str(self.startUserID)][self.targetItems[target]] = self.targetScore
                self.spamItem[str(self.startUserID)].append(self.targetItems[target])
            
            self.startUserID += 1
      
        print(self.spamProfile)

In [23]:
import pandas as pd

avg_attack = AverageAttack('/content/drive/My Drive/KCAMF/Attacks/config/config.conf')
avg_attack.insertSpam(startID = 5000)

attack_size = avg_attack.attackSize
target_count = avg_attack.targetCount

fname = 'TA_Avg_A_{}_T_{}.csv'.format(int(attack_size*100), target_count)
avg_attack.generateSpamProfiles(fname)

/content/drive/My Drive/KCAMF/Attacks/config/config.conf
loading training data...
Selecting target items...
Target item       Average rating of the item
666                    3.5
182                    3.125
1507                    3.3333333333333335
86                    3.2
1466                    3.375
305                    3.142857142857143
675                    3.4285714285714284
1284                    3.1666666666666665
1898                    3.5
716                    3.5
['666', '182', '1507', '86', '1466', '305', '675', '1284', '1898', '716', '666\n', '182\n', '1507\n', '86\n', '1466\n', '305\n', '675\n', '1284\n', '1898\n', '716\n']
Modeling average attack...
defaultdict(<class 'dict'>, {'5000': {'1566': 4.0, '1224': 5.0, '1477': 4.0, '2124': 4.0, '1627': 5.0, '1173': 4.0, '1837': 3.0, '1889': 4.0, '1920': 4.33, '1336': 4.29, '1481': 5.0, '205': 4.21, '236': 5.0, '53': 4.16, '1227': 4.0, '38': 3.0, '2148': 5.0, '1205': 4.0, '666': 5.0, '182\n': 5.0, '1507\n': 5.0, '1507'

# **Love/Hate Attack**

In [24]:
import random
import numpy as np

class LoveHateAttack(Attack):
    def __init__(self,conf):
        super(LoveHateAttack, self).__init__(conf)

    def insertSpam(self,startID=0):
        print('Modeling Love/Hate attack...')
        itemList = list(self.itemProfile.keys())
        #print(itemList)
        if startID == 0:
            self.startUserID = len(self.userProfile) + 1
        else:
            self.startUserID = startID

        for i in range(int(len(self.userProfile)*self.attackSize)):
            #fill 
            fillerItems = self.getFillerItems()
            #print(fillerItems)
            for item in fillerItems:
                self.spamProfile[str(self.startUserID)][str(itemList[item])] = 5.0
            
            #target 
            #print(self.targetItems)
            for j in range(self.targetCount):
                target = np.random.randint(len(self.targetItems))
                self.spamProfile[str(self.startUserID)][self.targetItems[target]] = 1.0
                self.spamItem[str(self.startUserID)].append(self.targetItems[target])
            
            self.startUserID += 1
      
        #print(self.spamProfile)

In [25]:
import pandas as pd

lh_attack = LoveHateAttack('/content/drive/My Drive/KCAMF/Attacks/config/config.conf')
lh_attack.insertSpam(startID = 5000)

attack_size = lh_attack.attackSize
target_count = lh_attack.targetCount

fname = 'TA_LH_A_{}_T_{}.csv'.format(int(attack_size*100), target_count)
lh_attack.generateSpamProfiles(fname)

/content/drive/My Drive/KCAMF/Attacks/config/config.conf
loading training data...
Selecting target items...
Target item       Average rating of the item
276                    3.4285714285714284
1520                    3.4285714285714284
1466                    3.375
182                    3.125
716                    3.5
305                    3.142857142857143
1898                    3.5
1284                    3.1666666666666665
666                    3.5
888                    2.7142857142857144
['276', '1520', '1466', '182', '716', '305', '1898', '1284', '666', '888', '276\n', '1520\n', '1466\n', '182\n', '716\n', '305\n', '1898\n', '1284\n', '666\n', '888\n']
Modeling Love/Hate attack...
Spam profiles have been output to /content/drive/My Drive/KCAMF/Attacks/attackDataset.


# **Post work**

In [26]:
import pandas as pd

ta = pd.read_csv("/content/drive/My Drive/KCAMF/Attacks/dataset/TripAdvisor/TA_train.csv")
ta.head()

,UserID,ItemID,Rating,TripType
0,1,1,5.0,SOLO
1,2,2,5.0,FAMILY
2,3,3,5.0,FRIENDS
3,4,4,5.0,FAMILY
4,5,5,4.0,COUPLES


In [27]:
fname = 'attackDataset/TA_LH_A_{}_T_{}.csv'.format(1, 1)
ta_lh_df = pd.read_csv(fname, header=None)
ta_lh_df.head()

,0,1,2
0,5000.0,328.0,5.0
1,5000.0,405.0,5.0
2,5000.0,457.0,5.0
3,5000.0,616.0,5.0
4,5000.0,951.0,5.0


In [28]:
at_size = [0.01, 0.03, 0.05, 0.1, 0.15, 0.2]
tar_count = [1, 5, 10]
cols = ['UserID', 'ItemID', 'Rating', 'TripType']

for attack_size in at_size:
  print(attack_size)
  for target_count in tar_count:
    print(target_count)
    fname = 'attackDataset/TA_LH_A_{}_T_{}.csv'.format(int(attack_size*100), target_count)
    ta_lh_df = pd.read_csv(fname, header=None)
    ta_lh_df_ctx = pd.DataFrame(columns=cols)
    len_profiles = len(ta_lh_df[ta_lh_df[0]!=0])
    for cnt in range(len_profiles):
      ta_lh_df_ctx.at[cnt, 'UserID'] = int(ta_lh_df.iloc[cnt,0])
      ta_lh_df_ctx.at[cnt, 'ItemID'] = int(ta_lh_df.iloc[cnt,1])
      ta_lh_df_ctx.at[cnt, 'Rating'] = ta_lh_df.iloc[cnt,2]

      z = ta[ta['ItemID']==int(ta_lh_df.iloc[0,1])].values
      ta_lh_df_ctx.at[cnt, 'TripType'] = z[0][3]
      
    ta_lh_df_ctx = pd.concat([ta_lh_df_ctx, ta])
    fname_ctx = 'attackDataset/TA_LH_Train_A_{}_T_{}.csv'.format(int(attack_size*100), target_count)
    ta_lh_df_ctx.to_csv(fname_ctx, index=False)    


0.01
1
5
10
0.03
1
5
10
0.05
1
5
10
0.1
1
5
10
0.15
1
5
10


IndexError: ignored

In [ ]:
at_size = [0.01, 0.03, 0.05, 0.1, 0.15, 0.2]
tar_count = [1, 5, 10]
cols = ['UserID', 'ItemID', 'Rating', 'TripType']

for attack_size in at_size:
  print(attack_size)
  for target_count in tar_count:
    print(target_count)
    fname = 'attackDataset/TA_Avg_A_{}_T_{}.csv'.format(int(attack_size*100), target_count)
    ta_avg_df = pd.read_csv(fname, header=None)
    ta_avg_df_ctx = pd.DataFrame(columns=cols)
    len_profiles = len(ta_avg_df[ta_avg_df[0]!=0])
    for cnt in range(len_profiles):
      ta_avg_df_ctx.at[cnt, 'UserID'] = int(ta_avg_df.iloc[cnt,0])
      ta_avg_df_ctx.at[cnt, 'ItemID'] = int(ta_avg_df.iloc[cnt,1])
      ta_avg_df_ctx.at[cnt, 'Rating'] = ta_avg_df.iloc[cnt,2]

      z = ta[ta['ItemID']==int(ta_avg_df.iloc[0,1])].values
      ta_avg_df_ctx.at[cnt, 'TripType'] = z[0][3]
      
    ta_avg_df_ctx = pd.concat([ta_avg_df_ctx, ta])
    fname_ctx = 'attackDataset/TA_Avg_Train_A_{}_T_{}.csv'.format(int(attack_size*100), target_count)
    ta_avg_df_ctx.to_csv(fname_ctx, index=False)    


0.01
1
5
10
0.03
1
5
10
0.05
1
5
10
0.1
1
5
10
0.15
1
5
10
0.2
1
5
10
